In [1]:
from ipywidgets import widgets
from IPython.display import display, display_svg, clear_output
from globus_automate_client import create_flows_client, graphviz_format, state_colors_for_log
import json
import time

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"
fc = create_flows_client(CLIENT_ID)

In [2]:
flows_resp = fc.list_flows()
for flow in flows_resp.data['flows']:
    print(f'Flow id: {flow["id"]}, Flow title: {flow["title"]}')
flows = {flow['id']:flow['title'] for flow in flows_resp.data['flows']}

Flow id: 06605825-9d12-4c98-8364-c0dd685d5cca, Flow title: Deployed via SDK
Flow id: 0a9a7296-f9ea-4a8a-9eb3-f95cf3e98edc, Flow title: Hello World Flow
Flow id: 0cd381f8-8abe-4189-8c4c-64ba9e6f98f6, Flow title: My Flow
Flow id: 11ba67a0-c5a5-491d-9a8a-766e4b0ba036, Flow title: Deployed via SDK
Flow id: 144b8f47-8ff8-40ea-8878-bbc0f71b8052, Flow title: My Flow
Flow id: 160e3141-03d7-4ba7-8231-c4c84514e6cd, Flow title: Deployed via SDK
Flow id: 1beb2248-77de-431f-acc2-0ff9b1882446, Flow title: My Flow
Flow id: 1dda16d4-9f0d-4239-8c5c-3210eedf82a0, Flow title: Deployed via SDK
Flow id: 1e6b4406-ee3d-4bc5-9198-74128e108111, Flow title: Hello World Flow
Flow id: 2d14922c-7cdb-429c-90d9-a42fb9632897, Flow title: Hello World
Flow id: 36039181-85de-4737-85a4-9bb28773f1a9, Flow title: Deployed via SDK
Flow id: 3e174e4c-094d-4f88-ae8d-a6db94b77832, Flow title: My Flow
Flow id: 49698e03-84af-4e3c-8d8e-a0db7c9c6f34, Flow title: Hello World Flow
Flow id: 49ee5587-cfe7-45a5-a202-f536e79c04f9, Flow t

In [ ]:
# Select an id from one of the flows above to continue exploring and running
flow_id = '830b860b-e98b-4186-8185-8f60d6bf25fb'

In [ ]:
get_resp = fc.get_flow(flow_id)
flow_def = get_resp.data['definition']
flow_graph = graphviz_format(flow_def)
display(flow_graph)

In [ ]:

text = widgets.Textarea(placeholder='Enter JSON formatted input for to the Flow', 
                        layout={'height': '100%', 'width': '500px'})
button = widgets.Button(description='Click to Run Flow')
items = [text, button]
box = widgets.HBox(items, layout={'height': '300px'})
display(box)
action_id = ''

def on_input(sender):
    input_val = text.value
    flow_input = json.loads(input_val)
    run_resp = fc.run_flow(flow_id, None, flow_input)
    global action_id
    action_id = run_resp.data['action_id']
    print(f'action_id: {action_id}')
    
button.on_click(on_input)

In [ ]:
while(True):
    status_resp  = fc.flow_action_status(flow_id, None, action_id)
    log_resp = fc.flow_action_log(flow_id, None, action_id, limit=100)
    state_colors = state_colors_for_log(log_resp.data['entries'])
    run_graph = graphviz_format(flow_def, state_colors)
    print(f'Action {action_id} is in state {status_resp.data["status"]} at time {time.ctime()}')
    display(run_graph)
    if status_resp.data['status'] in ('SUCCEEDED'):
        break
    time.sleep(5)
    clear_output(wait=True)